In [101]:
import pandas as pd
import numpy as np
import os
import sys
import gc
from sklearn.manifold import TSNE
from importlib import reload
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
import scipy 


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import utils
reload(utils)

<module 'utils' from '/Users/milk/Desktop/git_repositories/READMISSIONS/washtenaw_case_study/notebooks/utils.py'>

In [102]:
matplotlib.rcParams['figure.dpi'] = 200
plt.style.use('seaborn-deep')

In [103]:
"""
file paths
"""

GOOGLE_PATH = "../image_data/google_features.csv"
EIGEN_PATH = "../image_data/eigen_features.csv"
POPULATION_PATH = "../demographic_data/Population_Data_2015.csv"
EDUCATION_PATH = "../demographic_data/Education_Data_2015.csv"
INCOME_PATH = "../demographic_data/Income_Data_2015.csv"
OCCUPATION_PATH = "../demographic_data/Occupation_Data_2015.csv"
AA_500_CITIES_PATH = "../demographic_data/demographic_data/Ann_Arbor_500_Cities.csv"

# Google Features and Demographics

In [104]:
# """
# Load demographics paths anf filter by county
# """

pop_df = pd.read_csv(POPULATION_PATH)
# save short names and drop first row
POP_COLNAMES = pop_df.iloc[0].to_dict()
pop_df = pop_df.iloc[1:].reset_index()
pop_df[['Census Tract', 'County', 'State']] = pop_df['NAME'].str.split(',',expand=True)
pop_df['GEOID'] = pop_df['GEO_ID'].str.replace("1400000US", "").astype(int)
print(f"pop_df.shape {pop_df.shape}")
print()


edu_df = pd.read_csv(EDUCATION_PATH)
# save short names and drop first row
EDU_COLNAMES = edu_df.iloc[0].to_dict()
edu_df = edu_df.iloc[1:].reset_index()
edu_df[['Census Tract', 'County', 'State']] = edu_df['NAME'].str.split(',',expand=True)
edu_df['GEOID'] = edu_df['GEO_ID'].str.replace("1400000US", "").astype(int)
print(f"edu_df.shape {edu_df.shape}")
print()


inc_df = pd.read_csv(INCOME_PATH)
# save short names and drop first row
INC_COLNAMES = inc_df.iloc[0].to_dict()
inc_df = inc_df.iloc[1:].reset_index()
inc_df[['Census Tract', 'County', 'State']] = inc_df['NAME'].str.split(',',expand=True)
inc_df['GEOID'] = inc_df['GEO_ID'].str.replace("1400000US", "").astype(int)
print(f"inc_df.shape {inc_df.shape}")
print()

occ_df = pd.read_csv(OCCUPATION_PATH)
OCC_COLNAMES = occ_df.iloc[0].to_dict()
occ_df = occ_df.iloc[1:].reset_index()
occ_df[['Census Tract', 'County', 'State']] = occ_df['NAME'].str.split(',',expand=True)
occ_df['GEOID'] = occ_df['GEO_ID'].str.replace("1400000US", "").astype(int)
print(f"occ_df.shape {occ_df.shape}")
print()

pop_df.shape (2813, 343)

edu_df.shape (1416, 775)

inc_df.shape (1416, 127)

occ_df.shape (1416, 367)



In [105]:
%%time
"""
load google features and merge demographics
"""
data_df = pd.read_csv(GOOGLE_PATH)
data_df['GEOID'] = data_df['GEOID'].astype(int)
print(f"data_df.shape: {data_df.shape}")

data_df = pd.merge(data_df, pop_df, how='left', on='GEOID', suffixes=[None, "_pop"])
print(f"data_df.shape: {data_df.shape}")

data_df = pd.merge(data_df, edu_df, how='left', on='GEOID', suffixes=[None, "_edu"])
print(f"data_df.shape: {data_df.shape}")

data_df = pd.merge(data_df, inc_df, how='left', on='GEOID', suffixes=[None, "_inc"])
print(f"data_df.shape: {data_df.shape}")

data_df = pd.merge(data_df, occ_df, how='left', on='GEOID', suffixes=[None, "_occ"])
print(f"data_df.shape: {data_df.shape}")

data_df.shape: (5964, 729)
data_df.shape: (5964, 1071)
data_df.shape: (5964, 1845)
data_df.shape: (5964, 1971)
data_df.shape: (5964, 2337)
CPU times: user 716 ms, sys: 181 ms, total: 897 ms
Wall time: 926 ms


In [106]:
"""
drop other counties
"""

print(f"WARNING! Dropping non-Washtenaw county images")
data_df = data_df[data_df['County'] == ' Washtenaw County']
print(f"data_df.shape: {data_df.shape}")


WARNING! Dropping non-Washtenaw county images
data_df.shape: (5013, 2337)


In [107]:
%%time
"""
load eigen features
"""
eig_df = pd.read_csv(EIGEN_PATH)
print(eig_df.shape)

data_df = pd.merge(data_df, eig_df, left_on='image_index', right_on='tile_index', how='left')
print(data_df.shape)

(5964, 3001)
(5013, 5338)
CPU times: user 4.59 s, sys: 499 ms, total: 5.09 s
Wall time: 5.24 s


In [108]:
data_df.to_csv("../cleaned_data/clean_data.csv", index=False)
print("done")

done


In [109]:
colmaps = {
    'education' : EDU_COLNAMES, 
    'income' : INC_COLNAMES, 
    'occuptation' : OCC_COLNAMES,
    'population' : POP_COLNAMES
    }

for name, colmap in colmaps.items():
    colmap_df = pd.DataFrame(colmap.items())
    new_header = colmap_df.iloc[0] 
    colmap_df = colmap_df[1:] 
    colmap_df.columns = new_header 
    f_name = f"../cleaned_data/{name}_column_map.csv"
    colmap_df.to_csv(f_name, index=False)

In [83]:
# ALL_LABEL_COLS = [x for x in data_df.columns if 'label' in x]
# print(f"Number of Distinct Labels: {len(ALL_LABEL_COLS)}")

# MANUAL_LABELS = [x.strip() for x in open("../image_data/manual_labels.txt")]
# print(f"Number of Distinct Labels in curated list: {len(MANUAL_LABELS)}")

In [82]:
# REDS = [x for x in data_df.columns if 'red_eig' in x]
# BLUES = [x for x in data_df.columns if 'blue_eig' in x]
# GREENS = [x for x in data_df.columns if 'green_eig' in x]

In [71]:
# eigen_cols_df = pd.DataFrame([REDS, GREENS, BLUES]).T
# eigen_cols_df.columns = ['Red_Columns', 'Green_Columns', 'Blue_Columns']
# f_name = f"../cleaned_data/eigenvalue_column_map.csv"
# eigen_cols_df.to_csv(f_name, index=False)

In [81]:
# COLOR_R = []
# COLOR_G = []
# COLOR_B = []
# COLOR_SCORES = []
# COLOR_FRACTIONS = []

# LABELS = []
# LABEL_SCORES = []

# for col in data_df.columns:
#     if 'label' in col:
#         LABELS.append(col)
#     if 'score' in col and not 'COLOR' in col:
#         LABEL_SCORES.append(col)
#     if 'COLOR' in col and '_R' in col:
#         COLOR_R.append(col)
#     if 'COLOR' in col and '_G' in col:
#         COLOR_G.append(col)
#     if 'COLOR' in col and '_B' in col:
#         COLOR_B.append(col)
#     if 'COLOR' in col and 'score' in col:
#         COLOR_SCORES.append(col)
#     if 'COLOR' in col and 'fraction' in col:
#         COLOR_FRACTIONS.append(col)


# google_cols = pd.DataFrame([LABELS, LABEL_SCORES]).T
# google_cols.columns = ['Label_columns', 'Label_score_columns']
# google_cols.loc[:,'r_color_pixel_columns'] = pd.Series(COLOR_R)
# google_cols.loc[:,'g_color_pixel_columns'] = pd.Series(COLOR_G)
# google_cols.loc[:,'b_color_pixel_columns'] = pd.Series(COLOR_B)
# google_cols.loc[:,'color_score_columns'] = pd.Series(COLOR_SCORES)
# google_cols.loc[:,'color_fraction_columns'] = pd.Series(COLOR_FRACTIONS)

# f_name = f"../cleaned_data/google_column_map.csv"
# google_cols.to_csv(f_name, index=False)

In [87]:
# data_df = pd.read_csv("../cleaned_data/clean_data.csv")
# print(data_df.shape)

(5013, 4996)
